In [1]:
import argparse
from collections import Counter
import logging
import os

import cv2
import numpy as np
import pandas as pd
import torch

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/repos/mrnet/scripts')
from loader import load_data
from model_choice import MODELS
import cam as pycam

In [7]:
import loader3

In [8]:
# get case IDs
paths = pd.read_csv(
    '/mnt/mrnet-image-paths.csv', header=None, names=['path']
).path.values
path_df = loader3.paths_to_df(paths)
label_df = pd.read_csv('/mnt/mrnet-labels-3way.csv', index_col=0)
acl_test = loader3.Dataset(
    path_df=path_df, diagnosis='acl', label_df=label_df[label_df.split == 'test'], use_gpu=False
)
meniscus_test = loader3.Dataset(
    path_df=path_df, diagnosis='meniscus', label_df=label_df[label_df.split == 'test'], use_gpu=False
)

In [4]:
# Compare MRNet and SqueezeAttend on ACL (MRNet is better)

In [9]:
mrnet_preds = pd.read_csv('/mnt/ensemble_preds/MRNet_ensemble_preds.csv')
mrnet_preds.head()

,abnormal,abnormal_truth,acl,acl_truth,meniscus,meniscus_truth
0,0.230255,0.0,0.096320,0.0,0.137780,0.0
1,0.291873,0.0,0.125888,0.0,0.200391,0.0
2,0.363111,0.0,0.258895,0.0,0.180650,0.0
3,0.201513,0.0,0.144357,0.0,0.122091,0.0
4,0.500530,0.0,0.146775,0.0,0.275603,0.0


In [11]:
sa_preds = pd.read_csv('/mnt/ensemble_preds/MRNet-SqueezeAttend_ensemble_preds.csv')
sa_preds.head()

,abnormal,abnormal_truth,acl,acl_truth,meniscus,meniscus_truth
0,0.268497,0.0,0.269940,0.0,0.137336,0.0
1,0.241913,0.0,0.148453,0.0,0.171623,0.0
2,0.384587,0.0,0.339400,0.0,0.153698,0.0
3,0.233304,0.0,0.158560,0.0,0.127077,0.0
4,0.627901,0.0,0.228260,0.0,0.310481,0.0


In [17]:
acl_mrnet_correct = ((mrnet_preds.acl > 0.5) & (mrnet_preds.acl_truth == 1.0))
acl_sa_correct = ((sa_preds.acl > 0.5) & (sa_preds.acl_truth == 1.0))
acl_mrnet_miss = ((mrnet_preds.acl < 0.5) & (mrnet_preds.acl_truth == 1.0))
acl_sa_miss = ((sa_preds.acl < 0.5) & (sa_preds.acl_truth == 1.0))

print((acl_mrnet_correct & acl_sa_miss).sum())
print((acl_mrnet_miss & acl_sa_correct).sum())
print(acl_mrnet_correct.sum())
print(acl_sa_correct.sum())

5
2
49
46


In [18]:
print(mrnet_preds[acl_mrnet_correct & acl_sa_miss])
print(sa_preds[acl_mrnet_correct & acl_sa_miss])

    abnormal  abnormal_truth       acl  acl_truth  meniscus  meniscus_truth
43  0.928853             1.0  0.535112        1.0  0.470455             1.0
48  0.897948             1.0  0.753139        1.0  0.436394             0.0
66  0.954080             1.0  0.751383        1.0  0.716080             1.0
88  0.849023             1.0  0.589246        1.0  0.398551             0.0
98  0.839466             1.0  0.625173        1.0  0.707827             1.0
    abnormal  abnormal_truth       acl  acl_truth  meniscus  meniscus_truth
43  0.900401             1.0  0.299918        1.0  0.360570             1.0
48  0.883354             1.0  0.462831        1.0  0.541787             0.0
66  0.936284             1.0  0.498084        1.0  0.683574             1.0
88  0.780792             1.0  0.327571        1.0  0.262455             0.0
98  0.880340             1.0  0.422807        1.0  0.753436             1.0


In [19]:
# 88 looks good

In [20]:
acl_test.cases[88]

'1218.npy'

In [21]:
vols, label, case = acl_test[88]
axial, coronal, sagittal = vols

In [23]:
label

tensor([1.])

In [5]:
# Compare MTNet and SqueezeAttend on Meniscus (SqueezeAttend is better)